## **Install Required Python Libraries**

You are editing this notebook, so we assume you have already cloned the repository from  
[https://github.com/klinucsd/interagency-tracking-system](https://github.com/klinucsd/interagency-tracking-system) into the folder ~/work/interagency-tracking-system. Please verify that the `requirements.txt` file is available in that folder before proceeding.

In [1]:
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


## **Unzip Reference Data in GDB format**

In [2]:
! unzip -o ../a_Reference.gdb.zip 

Archive:  ../a_Reference.gdb.zip
  inflating: a_Reference.gdb/a00000007.CatRelTypesByBackwardLabel.atx  
  inflating: a_Reference.gdb/a0000000c.horizon  
  inflating: a_Reference.gdb/a0000000c.gdbtablx  
  inflating: a_Reference.gdb/a00000007.gdbtablx  
  inflating: a_Reference.gdb/timestamps  
  inflating: a_Reference.gdb/a00000010.spx  
  inflating: a_Reference.gdb/a00000011.I36Original_Acti.atx  
  inflating: a_Reference.gdb/a00000006.FDO_UUID.atx  
  inflating: a_Reference.gdb/a00000007.gdbtable  
  inflating: a_Reference.gdb/a00000004.gdbtablx  
  inflating: a_Reference.gdb/a00000010.freelist  
  inflating: a_Reference.gdb/a0000000b.gdbtablx  
  inflating: a_Reference.gdb/a00000001.gdbindexes  
  inflating: a_Reference.gdb/a00000009.horizon  
  inflating: a_Reference.gdb/a0000000b.gdbtable  
  inflating: a_Reference.gdb/a00000011.gdbtable  
  inflating: a_Reference.gdb/a0000000a.horizon  
  inflating: a_Reference.gdb/a00000003.gdbtable  
  inflating: a_Reference.gdb/a00000010.hori

## **Download USFS data**

In [3]:
from utils.download_from_box import download_box_shared_link

region04_box_link = 'https://calfire.box.com/s/1ijmb41oguc08ab5q70ph0i55hqu1q1y'
download_box_shared_link(region04_box_link)

region05_box_link = 'https://calfire.box.com/s/z6z4ylf7c0bcl5kldjle21zz86t6xgk7'
download_box_shared_link(region05_box_link)

region06_box_link = 'https://calfire.box.com/s/3f2sp5uvjihhe42o5h23evkq9ejx4x67'
download_box_shared_link(region06_box_link)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: Actv_CommonAttribute_PL_Region04_20250331.zip
    ✓ Saved: Actv_CommonAttribute_PL_Region04_20250331.zip (218,899,346 bytes)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: Actv_CommonAttribute_PL_Region05_20250331.zip
    ✓ Saved: Actv_CommonAttribute_PL_Region05_20250331.zip (526,488,841 bytes)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: Actv_CommonAttribute_PL_Region06_20250331.zip
    ✓ Saved: Actv_CommonAttribute_PL_Region06_20250331.zip (694,982,056 bytes)



['./Actv_CommonAttribute_PL_Region06_20250331.zip']

## **Unzip USFS data**

In [5]:
! unzip -o Actv_CommonAttribute_PL_Region04_20250331.zip
! unzip -o Actv_CommonAttribute_PL_Region05_20250331.zip
! unzip -o Actv_CommonAttribute_PL_Region06_20250331.zip

Archive:  Actv_CommonAttribute_PL_Region04_20250331.zip
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000001.gdbindexes  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000001.gdbtable  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000001.gdbtablx  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000001.TablesByName.atx  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000002.gdbtable  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000002.gdbtablx  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000003.gdbindexes  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000003.gdbtable  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000003.gdbtablx  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000004.CatItemsByPhysicalName.atx  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000004.CatItemsByType.atx  
  inflating: Actv_CommonAttribute_PL_Region04.gdb/a00000004.FDO_UUID.atx  
  inflating: Actv_CommonAttribute_PL_Region04.gd

## **Enriching USFS Data**

Converts the U.S. Forest Service actual treatments dataset into the Task Force standardized schema. Dataset is enriched with vegetation, ownership, county, WUI, Task Force Region, and year. 

In [ ]:

import warnings
import logging
import time
from datetime import datetime
import psutil
import os
from enrich.enrich_USFS import enrich_USFS

# Get the current process ID
process = psutil.Process(os.getpid())

a_reference_gdb_path = "a_Reference.gdb"
start_year, end_year = 1950, 2025
output_gdb_path = f"/tmp/USFS_{start_year}_{end_year}.gdb"

region_ids = ["04", "05", "06"]
for region_id in region_ids:    
    usfs_input_gdb_path = f"Actv_CommonAttribute_PL_Region{region_id}.gdb"
    usfs_input_layer_name = "Actv_CommonAttribute_PL"
    output_layer_name = f"USFS_Region{region_id}_enriched_{datetime.today().strftime('%Y%m%d')}"
    enrich_USFS(usfs_input_gdb_path,
                usfs_input_layer_name,
                a_reference_gdb_path,
                start_year,
                end_year,
                output_gdb_path,
                output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
logger.info(f"Memory usage: {memory_usage:.2f} MB")

2025-11-21 05:54:32,155 INFO  [enrich.enrich_USFS    ]  Loading the USFS data into GeoDataFrames: Actv_CommonAttribute_PL_Region04.gdb : Actv_CommonAttribute_PL
2025-11-21 05:54:32,156 INFO  [enrich.enrich_USFS    ]     Loading USFS data from source and cache the data
2025-11-21 05:55:28,195 INFO  [enrich.enrich_USFS    ]        records: 337783
2025-11-21 05:55:28,196 INFO  [enrich.enrich_USFS    ]        time for loading USFS: 56.03998851776123
2025-11-21 05:55:28,197 INFO  [enrich.enrich_USFS    ]     all required columns are present.
2025-11-21 05:55:37,816 INFO  [enrich.enrich_USFS    ]  Performing Standardization...
2025-11-21 05:55:38,040 INFO  [enrich.enrich_USFS    ]     found 137057 rows with empty geometry
2025-11-21 05:55:38,041 INFO  [enrich.enrich_USFS    ]     drop 137057 rows with empty geometry
2025-11-21 05:55:39,338 INFO  [enrich.enrich_USFS    ]     records in California: 979
2025-11-21 05:55:39,340 INFO  [enrich.enrich_USFS    ]     step 1/8 Selecting Features...
20